In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===== ENGLISH VOCAB =====
en_vocab = {
    0: torch.tensor([1.1, -1.0]),    # <eos>
    1: torch.tensor([-1.5, 0.6]),    # I
    2: torch.tensor([-0.8, -1.3]),   # hate
    3: torch.tensor([0.3, 1.1])      # you
}

# ===== VIETNAMESE VOCAB =====
vi_vocab = {
    0: torch.tensor([-2.3, -0.3]),   # <sos>
    1: torch.tensor([0.0, 0.0]),     # <eos>
    2: torch.tensor([-0.2, -2.4]),   # tôi
    3: torch.tensor([0.1, 2.1]),     # ghét
    4: torch.tensor([0.1, 0.0]),     # bạn
    5: torch.tensor([0.0, 0.2]),     # em
    6: torch.tensor([-0.1, -0.2])    # anh
}

# ===== ENCODER =====
class EncoderRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_ih = nn.Parameter(torch.tensor(
            [[-0.1, 1.1], [0.2, -1.1]]))  # 2x2
        self.b_ih = nn.Parameter(torch.tensor([-0.5, 0.7]))
        self.W_hh = nn.Parameter(torch.tensor(
            [[-0.2, 1.1], [0.6, -0.2]]))  # 2x2
        self.b_hh = nn.Parameter(torch.tensor([0.3, 0.0]))

    def forward(self, x_seq):
        h = torch.zeros(2)
        for x in x_seq:
            h = torch.tanh(self.W_ih @ x + self.b_ih +
                           self.W_hh @ h + self.b_hh)
        return h

# ===== DECODER =====
class DecoderRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_ih = nn.Parameter(torch.tensor([[0.7, -1.3], [-0.9, -0.8]]))
        self.b_ih = nn.Parameter(torch.tensor([0.0, -0.9]))
        self.W_hh = nn.Parameter(torch.tensor([[0.0, -0.1], [-0.3, 1.4]]))
        self.b_hh = nn.Parameter(torch.tensor([-0.3, -0.7]))

        self.fc = nn.Linear(2, 7)
        with torch.no_grad():
            self.fc.weight.copy_(torch.tensor([
                [0.6, 0.3],
                [-0.4, -0.8],
                [-1.0, 1.7],
                [1.8, 1.7],
                [0.9, 0.2],
                [0.8, 0.3],
                [-1.8, -0.4]
            ]))
            self.fc.bias.copy_(torch.tensor(
                [-0.3, 0.4, 0.8, 0.7, -1.0, -1.2, 0.1]))

    def forward(self, x_t, h_prev):
        h = torch.tanh(self.W_ih @ x_t + self.b_ih +
                       self.W_hh @ h_prev + self.b_hh)
        logits = self.fc(h)
        probs = F.softmax(logits, dim=-1)
        return probs, h

### FNE3MT01 

In [3]:
encoder = EncoderRNN()
x_seq = [en_vocab[1], en_vocab[2]]  # I hate you
h2 = encoder(x_seq)
print("Encoder hidden state h2:", h2)

Encoder hidden state h2: tensor([-0.9594,  0.9819], grad_fn=<TanhBackward0>)


### FNE3MT02